In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
import time
%matplotlib qt

# Part 1
Some constants

In [ ]:
charge = -1.9e-10 # coulombs

initial_vx = 20000 # mm/s
initial_vy = 0 # mm/s
initial_vz = 0 # mm/s
initial_x = 0 # Position, MM
initial_y = 0 # Position, MM
initial_z = 107.95 # Position, MM
initial_t = 0 # Seconds

droplet_diameter = 84e-3 # Meteres
droplet_density = 1000 # kg/m^3
droplet_mass = ( (4/3) * (droplet_diameter/2) ** 3) * math.pi * droplet_density
print_resolution = 300 # dpi

total_distance_travel = 3 # MM
plate_voltage = 0 # Volts, for part 1
plate_seperation = 1 # MM
plate_start = 125 # Position MM
plate_length = 0.5 # MM
plate_end = plate_start + plate_length # Position, MM
paper_end = 3 # MM

gravity_const = -9800 # MM/s^2


## Calculations For Question 1
We are assuming no voltage is applied to the plates. <br>
We are looking for how long it takes for the droplet to reach the paper. <br>
Due to the droplet being so small, we can negate air resistance. Therefore, the velocity will remain constant for both X and Y, for this case. <br>
While we have declared a part for the Z component, we will not use it in this case, as there will be zero force applied and zero position changed in the Z direction (for question 1)<br>

In [ ]:
droplet = ({'position_x': [initial_x], 'position_y': [initial_y], 'position_z': [initial_z],
                'velocity_x': [initial_vx], 'velocity_y': [initial_vy], 'velocity_z': [0],
                'time': [initial_t], 'combined_velocity': [initial_vx]})

electric_field = plate_voltage / plate_seperation # Should be zero in this case, due to no plate voltage
grav_force = gravity_const * droplet_mass # Gravitational force, which is always constant


force_z = 0 # 2 parallel plates, Z force not possible (in this portion)
step = 0
time_step = 1e-6

while (droplet['position_x'][step] <= paper_end):
    droplet['time'].append( time_step * (step + 1) )

    if (droplet['position_x'][step] < plate_start or droplet['position_x'][step] > plate_end): # Checks if the droplet is within the plates
        force_y = 0
    else: # If it is between the plates
        force_y = electric_field * charge # This should be zero in this case, due to there being no electric field between the plates

    droplet['velocity_x'].append(initial_vx) # Should not vary here
    droplet['velocity_z'].append( ( (force_y * time_step) / droplet_mass) + droplet['position_y'][step] + grav_force )
    droplet['velocity_y'].append(initial_vz)

    droplet['position_z'].append( (droplet['velocity_y'][step] * time_step) + droplet['position_z'][step] )
    droplet['position_x'].append( (initial_vx * time_step) + droplet['position_x'][step] )
    droplet['position_y'].append(0) # It is known it will not change in this scenario
    combined_velocity = math.sqrt( (droplet['velocity_x'][step] ** 2) + (droplet['velocity_y'][step] ** 2) + (droplet['velocity_z'][step] ** 2))

    droplet['combined_velocity'].append(combined_velocity) 
    
    step = step + 1
print(droplet['time'])
print(droplet['position_z'])


## Plotting Question 1

In [ ]:
fig1 = plt.figure(figsize=(8, 6))
ax1 = fig1.add_subplot(111, projection='3d')
ax1.set_title('Droplet trajectory when there is no voltage accross the plates')
ax1.set_xlabel('X Position (MM)')
ax1.set_zlabel('Z Position (MM)')
ax1.set_ylabel('Y Position (MM)')
ax1.set_ylim(np.min(droplet['position_y']) - 1, np.max(droplet['position_y']) + 1)
ax1.set_xlim(np.min(droplet['position_x']), np.max(droplet['position_x']))
ax1.set_zlim(np.min(droplet['position_z']) - 1, np.max(droplet['position_z']) + 1)
ax1.view_init(elev=0, azim=-110)

x_range = np.linspace(1.25, 1.75, 10)
z_range = np.linspace(107, 108, 10)

# 4. Generate a grid of X and Y values
X, Z = np.meshgrid(x_range, z_range)

# 5. Define the Z value as a constant (e.g., z=5)
Y = np.full(X.shape, 1)


# fig2 = plt.figure(figsize=(8, 6))
# ax2 = fig2.add_subplot(111)
# ax2.set_title("Velocity over time")
# ax2.set_ylabel('Velocity (m/s)')
# ax2.set_xlabel('Time (s)')
# ax2.set_ylim(np.min(droplet['combined_velocity']) - 10 , np.max(droplet['combined_velocity']) + 10)
# ax2.set_xlim(np.min(droplet['time']), np.max(droplet['time']))

dot1 = ax1.scatter([], [], [], color='r', label='Position')
# dot2 = ax2.scatter([], [], color='b', label='Velocity')

def update_position(frame):
    dot1._offsets3d = (droplet['position_x'][frame:frame+1], droplet['position_y'][frame:frame+1], droplet['position_z'][frame:frame+1])
    return dot1,

# def update_velocity(frame):
#     dot2.set_offsets(np.c_[droplet['time'][frame:frame+1], droplet['combined_velocity'][frame:frame+1]])
#     return dot2,

ani1 = animation.FuncAnimation(fig1, update_position, frames=step, interval=10, blit=False)
# ani2 = animation.FuncAnimation(fig2, update_velocity, frames=step, interval=10, blit=False)


plt.show()

# Calculating time it took to to reach the paper
print(f"The time it took to reach the paper was {droplet['position_x'][-1]} seconds")


# Question 2
Simulate the act of drawing a vertical line (similar to the letter “I” ) along the center of the paper at 300 dpi resolution. How much time would it take to draw the letter ‘I’? <br>
In this case, the voltage needs to be dynamically varied accross each step.

In [ ]:
droplet = ({'position_x': [initial_x], 'position_y': [initial_y], 'position_z': [initial_z],
                'velocity_x': [initial_vx], 'velocity_y': [initial_vy], 'velocity_z': [0],
                'time': [initial_t], 'combined_velocity': [initial_vx]})

electric_field = plate_voltage / plate_seperation # Should be zero in this case, due to no plate voltage
grav_force = gravity_const * droplet_mass # Gravitational force, which is always constant

previous_vy = initial_vy
previous_vx = initial_vx
previous_px = initial_x
previous_py = initial_y

force_z = 0 # 2 parallel plates, Z force not possible (in this portion)
step = 0
time_step = 1e-6

while (droplet['position_x'][step] <= paper_end):
    droplet['time'].append( time_step * (step + 1) )

    if (previous_px < plate_start or previous_px > plate_end): # Checks if the droplet is within the plates
        force_y = 0
    else: # If it is between the plates
        force_y = electric_field * charge # This should be zero in this case, due to there being no electric field between the plates

    droplet['velocity_x'].append(initial_vx) # Should not vary here
    droplet['velocity_y'].append( ( (force_y * time_step) / droplet_mass) + droplet['position_y'][step] + grav_force )

    droplet['position_y'].append( (droplet['velocity_y'][step] * time_step) + previous_py )
    droplet['position_x'].append( (initial_vx * time_step) + droplet['position_x'][step] )
    combined_velocity = math.sqrt( (droplet['velocity_x'][step] ** 2) + (droplet['velocity_y'][step] ** 2) )

    droplet['combined_velocity'].append(combined_velocity) 
    
    step = step + 1
print(droplet['time'])
print(droplet['position_x'])
print(len(droplet['time']),len(droplet['position_x']))